### Requirements to generate german credit multiclass csv

- keras (tensorflow backend)
- numpy
- certifai_common(encoder for model training)
- scikit-learn

In [1]:
import keras as keras_version_check
import tensorflow as tf_version_check
print(f'keras version {keras_version_check.__version__}')
print(f'tensorflow version {tf_version_check.__version__}')

Using TensorFlow backend.
/Users/akumar/miniconda/envs/hsbc-v1.3.3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/akumar/miniconda/envs/hsbc-v1.3.3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/akumar/miniconda/envs/hsbc-v1.3.3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/U

keras version 2.3.1
tensorflow version 1.14.0


/Users/akumar/miniconda/envs/hsbc-v1.3.3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/akumar/miniconda/envs/hsbc-v1.3.3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/akumar/miniconda/envs/hsbc-v1.3.3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import random
from certifai.common.utils.encoding import CatEncoder
from keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
from keras.models import Sequential
from keras import regularizers
from keras import optimizers
from keras.layers import Dense, Dropout
import os
import random
import tensorflow as tf
seed = 42

os.environ['PYTHONHASHSEED']=str(seed)
np.random.seed(seed)
random.seed(seed)

try:
    tf.random.set_seed(seed)
except AttributeError:
    tf.set_random_seed(seed)

base_path = '../..'
all_data_file = f"{base_path}/datasets/german_credit_eval.csv"

df = pd.read_csv(all_data_file)

cat_columns = [
    'checkingstatus',
    'history',
    'purpose',
    'savings',
    'employ',
    'status',
    'others',
    'property',
    'age',
    'otherplans',
    'housing',
    'job',
    'telephone',
    'foreign'
    ]

label_column = 'outcome'

# Separate outcome
y = df[label_column]
X = df.drop(label_column, axis=1)

encoder = CatEncoder(cat_columns, X)

# NN separate for probability scoring label 
y_model = np.asarray([(1,0) if x==1 else (0,1) for x in y ])

best_model_path = os.path.join('.', 'best_model_keras')
es = EarlyStopping(monitor='accuracy', mode='min', verbose=1,
                   patience=100, min_delta=0.0001)

rlp = ReduceLROnPlateau(monitor='accuracy', factor=0.02, patience=20, verbose=1, mode='min',
                        min_delta=0.001, cooldown=1, min_lr=0.0001)
mcp = ModelCheckpoint(best_model_path, monitor='accuracy', verbose=1,
                      save_best_only=True, save_weights_only=False, mode='max')

model = Sequential()
model.add(Dense(60, input_dim=encoder(X.values).shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(encoder(X.values), y_model, epochs=100, batch_size=10, callbacks=[mcp,rlp,es],validation_split=0.2)
_, accuracy = model.evaluate(encoder(X.values), y_model)
print('Accuracy: %.2f' % (accuracy*100))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 0s 466us/step - loss: 0.5672 - accuracy: 0.7100 - val_loss: 0.5189 - val_accuracy: 0.7650

Epoch 00001: accuracy improved from -inf to 0.71000, saving model to ./best_model_keras
Epoch 2/100
800/800 [==============================] - 0s 343us/step - loss: 0.4861 - accuracy: 0.7625 - val_loss: 0.4781 - val_accuracy: 0.7700

Epoch 00002: accuracy improved from 0.71000 to 0.76250, saving model to ./best_model_keras
Epoch 3/100
800/800 [==============================] - 0s 252us/step - loss: 0.4502 - accuracy: 0.7825 - val_loss: 0.4692 - val_accuracy: 0.8050

Epoch 00003: accuracy improved from 0.76250 to 0.78250, saving model to ./best_model_keras
Epoch 4/100
800/800 [==============================] - 0s 165us/step - loss: 0.4264 - accuracy: 0.8112 - val_loss: 0.4837 - val_accuracy: 0.7550

Epoch

800/800 [==============================] - 0s 118us/step - loss: 0.0775 - accuracy: 0.9875 - val_loss: 0.6866 - val_accuracy: 0.7750

Epoch 00034: accuracy did not improve from 0.98875
Epoch 35/100
800/800 [==============================] - 0s 127us/step - loss: 0.0762 - accuracy: 0.9887 - val_loss: 0.6892 - val_accuracy: 0.7750

Epoch 00035: accuracy did not improve from 0.98875
Epoch 36/100
800/800 [==============================] - 0s 138us/step - loss: 0.0756 - accuracy: 0.9862 - val_loss: 0.6912 - val_accuracy: 0.7750

Epoch 00036: accuracy did not improve from 0.98875
Epoch 37/100
800/800 [==============================] - 0s 162us/step - loss: 0.0748 - accuracy: 0.9887 - val_loss: 0.6976 - val_accuracy: 0.7650

Epoch 00037: accuracy did not improve from 0.98875
Epoch 38/100
800/800 [==============================] - 0s 134us/step - loss: 0.0736 - accuracy: 0.9887 - val_loss: 0.6983 - val_accuracy: 0.7750

Epoch 00038: accuracy did not improve from 0.98875
Epoch 39/100
800/800 [=


Epoch 00074: accuracy did not improve from 0.99375
Epoch 75/100
800/800 [==============================] - 0s 138us/step - loss: 0.0424 - accuracy: 0.9937 - val_loss: 0.8016 - val_accuracy: 0.7650

Epoch 00075: accuracy did not improve from 0.99375
Epoch 76/100
800/800 [==============================] - 0s 145us/step - loss: 0.0419 - accuracy: 0.9925 - val_loss: 0.8116 - val_accuracy: 0.7600

Epoch 00076: accuracy did not improve from 0.99375
Epoch 77/100
800/800 [==============================] - 0s 165us/step - loss: 0.0412 - accuracy: 0.9937 - val_loss: 0.8065 - val_accuracy: 0.7700

Epoch 00077: accuracy did not improve from 0.99375
Epoch 78/100
800/800 [==============================] - 0s 152us/step - loss: 0.0405 - accuracy: 0.9937 - val_loss: 0.8171 - val_accuracy: 0.7600

Epoch 00078: accuracy did not improve from 0.99375
Epoch 79/100
800/800 [==============================] - 0s 142us/step - loss: 0.0400 - accuracy: 0.9937 - val_loss: 0.8181 - val_accuracy: 0.7650

Epoch 000

In [3]:
# initial value counts of outcome label
df['outcome'].value_counts()

1    700
2    300
Name: outcome, dtype: int64

In [4]:
y_preds = model.predict(encoder(X.values))
y_classes = np.argmax(y_preds,axis=1)
y_actual = np.where(y==1, 0,1)
np.mean(y_actual == y_classes)

0.951

### creating new label

In [5]:
# loan_granted 1
new_label_1 = y_preds[y_preds[:,0] > 0.98][:,0]
new_label_1.shape

(539,)

In [6]:
#loan_denied 2
new_label_2 = y_preds[y_preds[:,0] < 0.3][:,0]
new_label_2.shape

(285,)

In [7]:
# further_inspection 3
new_label_3 = y_preds[(y_preds[:,0] > 0.3) & (y_preds[:,0] < 0.98)][:,0]
new_label_3.shape

(176,)

### arrange the new labels to be appended to dataset

In [8]:
new_label = []
for z in y_preds:
    prob_class_1 = z[0]
    if prob_class_1 > 0.98:
        new_label.append(1)
    elif prob_class_1 < 0.3:
        new_label.append(2)
    else:
        new_label.append(3)

In [9]:
# test for sanity of new labels
np.mean(y==new_label)

0.795

### create dataset with new labels

In [10]:
df_new = df.copy()
df_new[label_column] = new_label

### check new outcome label counts

In [11]:
df_new['outcome'].value_counts()

1    539
2    285
3    176
Name: outcome, dtype: int64

### dump the new dataset as csv

In [12]:
save_as = f"{base_path}/datasets/german_credit_eval_multiclass.csv"
df_new.to_csv(save_as,index=False)

### check file is saved correctly

In [13]:
!ls $base_path/datasets | grep multiclass.csv

german_credit_eval_multiclass.csv
